# **Scraping Data - Coinbase**

**Deskripsi:**
Mengambil data ulasan (review) aplikasi Coinbase dari Google Play Store. Jika data mentah sudah ada dalam file CSV, data akan dimuat dari file tersebut untuk menghindari proses scraping ulang.

**Proses:**
1.  Menentukan konfigurasi scraping: ID Aplikasi (`com.coinbase.android`), bahasa (`en`), negara (`us`), target jumlah review (`300000`), dan path file CSV untuk menyimpan data mentah (`data/reviews_1_raw.csv`).
2.  Mengecek apakah file `RAW_CSV_PATH` sudah ada.
3.  **Jika tidak ada:** Melakukan scraping menggunakan `reviews_all` dari `google_play_scraper`, mengukur waktu scraping, mengubah hasilnya menjadi DataFrame Pandas, memilih kolom relevan (`reviewId`, `userName`, `content`, `score`, `thumbsUpCount`, `reviewCreatedVersion`, `at`), mengganti nama kolom `content` menjadi `review_text` dan `score` menjadi `rating`, lalu menyimpan ke file CSV.
4.  **Jika sudah ada:** Memuat data langsung dari file CSV menggunakan `pd.read_csv`.

---

In [1]:
# !pip install google-play-scraper

In [2]:
import os
import time
import pandas as pd
from google_play_scraper import reviews_all, Sort, reviews

In [3]:
# Konfigurasi
APP_ID = 'com.coinbase.android'
LANG = 'en'
COUNTRY = 'us'
NUM_REVIEWS_TARGET = 300000
RAW_CSV_PATH = 'data/reviews_1_raw.csv'

In [4]:
# --- Scraping ---
if not os.path.exists(RAW_CSV_PATH):
    print(f"Memulai scraping review untuk {APP_ID}...")
    start_time = time.time()
    try:
        all_reviews = reviews_all(
            APP_ID,
            lang=LANG,
            country=COUNTRY,
            sort=Sort.NEWEST,
            count=NUM_REVIEWS_TARGET
        )
        
        df_raw = pd.DataFrame(all_reviews)
        print(f"Scraping selesai. Mendapatkan {len(df_raw)} review.")
        # Ambil kolom yang relevan saja
        df_raw = df_raw[['reviewId', 'userName', 'content', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at']]
        df_raw.rename(columns={'content': 'review_text', 'score': 'rating'}, inplace=True)
        df_raw.to_csv(RAW_CSV_PATH, index=False)
        print(f"Data mentah disimpan ke {RAW_CSV_PATH}")
    except Exception as e:
        print(f"Terjadi error saat scraping: {e}")
        df_raw = pd.DataFrame() # Buat dataframe kosong jika gagal
    end_time = time.time()
    print(f"Waktu scraping: {end_time - start_time:.2f} detik.")
else:
    print(f"Memuat data mentah dari {RAW_CSV_PATH}...")
    df_raw = pd.read_csv(RAW_CSV_PATH)
    print(f"Berhasil memuat {len(df_raw)} review mentah.")

Memuat data mentah dari data/reviews_1_raw.csv...
Berhasil memuat 172310 review mentah.


### **Insight:**
Berhasil mengumpulkan 172.310 review mentah dari Google Play Store dalam waktu sekitar 3 menit lebih. Jumlah ini kurang dari target 300.000, kemungkinan karena batasan dari scraper atau jumlah review yang tersedia saat itu. Data mentah disimpan dalam format CSV (`df_raw`) untuk digunakan pada fase berikutnya dan mencegah scraping berulang. Kolom-kolom penting telah dipilih dan dinamai ulang untuk konsistensi.